In [9]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import time, random

import json
import re
import os

job_links = []

# Header giả lập trình duyệt thật
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/119.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    # 👇 Tắt yêu cầu nén gzip/br
    "Accept-Encoding": "identity",
    "Accept-Language": "vi-VN,vi;q=0.9,en;q=0.8",
    "Connection": "keep-alive",
}

def crawl_page(page):
    url = f"https://www.topcv.vn/viec-lam-it?page={page}"
    try:
        # Delay ngẫu nhiên tránh bị chặn
        time.sleep(random.uniform(2, 4))

        response = requests.get(url, headers=headers, timeout=15)
        response.encoding = response.apparent_encoding  # tự đoán encoding

        # Lưu HTML của trang ra file để kiểm tra
        #with open(f"page_{page}.html", "wb") as f:
            #f.write(response.content)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            all_job = soup.find_all("div", class_="job-item-2")
            links = []
            for job in all_job:
                a_tag = job.find("a", href=True)
                if a_tag:
                    links.append(a_tag["href"])
            # print(f"Trang {page}: {len(links)} links")
            return links
        else:
            print(f"Trang {page} lỗi HTTP {response.status_code}")
    except Exception as e:
        print(f"Lỗi trang {page}: {e}")
    return []


# === Chạy đa luồng nhẹ (đừng quá cao nếu không bị chặn) ===
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    results = executor.map(crawl_page, range(2, 22))  # thử trước 20 trang
    for r in results:
        job_links.extend(r)

print("\nTổng số link thu được:", len(job_links))


Tổng số link thu được: 1000


In [8]:
job_links[0]

IndexError: list index out of range

In [ ]:
def link_to_json(url,number_of_file):
    #Lấy soup
    response = requests.get(url, headers= headers, timeout = 15)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        # print(soup)

    #Lấy job_description_content
    job_description_content = soup.find("div", class_=lambda c: c and "job-description" in c)
    # print(job_description_content)
    job_text = job_description_content.get_text(separator="\n",strip=True)
    # print(job_text)

    # text = extract_job_info(soup)
    output_lines = []
    print("=== Tên công ty ===")
    company_name_label = soup.find("div", class_ = "company-name-label")
    if ( company_name_label ):
        a_tag = company_name_label.find("a")
        if a_tag:
            company_name = a_tag.get_text(strip=True)
            output_lines.append("=== Tên công ty ===")
            output_lines.append(company_name)

    job_detail_info = soup.find_all("div", class_="job-detail__info--section")
    for section in job_detail_info:
        title_tag = section.find("div", class_="job-detail__info--section-content-title")
        value_tag = section.find("div", class_="job-detail__info--section-content-value")

        if value_tag:
            a_tag = value_tag.find("a")
            if a_tag:
                value_text = a_tag.get_next(strip=True)
            else:
                value_text = value_tag.get_next(strip=True)
        else:
            value_text = ""

        title_text = title_tag.get_next(strip=True) if title_tag else "Không có tiêu đề"

        output_lines.append(f"=== {title_text} ===")
        output_lines.append(value_text)

    items = job_description_content.find_all("div", class_ = "job-description__item")
    for item in items:
        title = item.find("h3").get_text(strip=True)
        content = item.find("div", class_="job-description__item--content").get_text(separator="\n", strip=True)
        output_lines.append(f"=== {title} ===")
        output_lines.append(content)
    
    text = "\n".join(output_lines)
    # text_to_json(text, TÊN_FILE_Ở_ĐÂY)
    print(text)